# Plot Description Based Recommender

In [1]:
import pandas as pd
import numpy as np

#Import data from the clean file 
df = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/metadata_clean.csv')

#Print the head of the cleaned DataFrame
df.head()

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995


In [2]:
df.shape

(45466, 6)

In [3]:
#Import the original file
orig_df = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/movies_metadata.csv', low_memory=False)
print(orig_df.shape)
orig_df.head()

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
#Add the useful features into the cleaned dataframe
df['overview'], df['id'] = orig_df['overview'], orig_df['id']
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['animation', 'comedy', 'family']",81.0,7.7,5415.0,1995,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['adventure', 'fantasy', 'family']",104.0,6.9,2413.0,1995,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['romance', 'comedy']",101.0,6.5,92.0,1995,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['comedy', 'drama', 'romance']",127.0,6.1,34.0,1995,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['comedy'],106.0,5.7,173.0,1995,Just when George Banks has recovered from his ...,11862


In [5]:
#Replace NaN with an empty string
df['overview'] = df['overview'].fillna('')
print(df.shape) # Nen giam xuong con 9000 thoi thi ok

(45466, 8)


## DNN giam dữ liệu movie xuống còn 9000 phim ngẫu nhie6n

In [6]:
df=df.sample(frac=0.2,replace=False, random_state=1) #20% sample of the dataframe with replacement
df.shape

(9093, 8)

In [7]:
df=df.drop_duplicates(subset='title', keep="first")
df.shape

(8924, 8)

In [8]:
df.reset_index(drop=True,inplace=True)

In [9]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,The New Comics,['comedy'],102.0,5.0,32.0,1994,A few funny little novels about different aspe...,43649
1,China Girl,"['romance', 'crime', 'drama']",90.0,6.6,8.0,1987,Teenage lovers Tony (Richard Panebianco) and T...,47212
2,Snow Shark: Ancient Snow Beast,"['science fiction', 'horror']",80.0,4.0,2.0,2011,12 years ago during a scientific expedition 3 ...,133953
3,Penny Serenade,"['drama', 'romance']",119.0,6.4,23.0,1941,A light-hearted romantic drama starring Cary G...,43795
4,Iron And Silk,"['drama', 'comedy']",92.0,6.0,1.0,1991,Iron and Silk is a 1990 movie based on the epo...,23981


In [10]:
df[df.title=='Revenge']

,title,genres,runtime,vote_average,vote_count,year,overview,id
8509,Revenge,['drama'],100.0,5.6,5.0,1989,"Enraged, a teacher murders a young female pupi...",216046


In [11]:
#Import TfIdfVectorizer from the scikit-learn library
from sklearn.feature_extraction.text import TfidfVectorizer

#Define a TF-IDF Vectorizer Object. Remove all english stopwords
tfidf = TfidfVectorizer(stop_words='english',#max_features=20000
                        )
#Construct the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(df['overview'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

(8924, 33163)

In [12]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [14]:
print(cosine_sim.shape)
cosine_sim

(8924, 8924)


array([[1.        , 0.04785325, 0.        , ..., 0.        , 0.01708081,
        0.        ],
       [0.04785325, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.02211116, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.02211116, ..., 1.        , 0.01225323,
        0.        ],
       [0.01708081, 0.        , 0.        , ..., 0.01225323, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [15]:
#Construct a reverse mapping of indices and movie titles, and drop duplicate titles, if any
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [16]:
indices

title
The New Comics                       0
China Girl                           1
Snow Shark: Ancient Snow Beast       2
Penny Serenade                       3
Iron And Silk                        4
                                  ... 
Kicks                             8919
Resident Evil                     8920
All That Heaven Allows            8921
Badmaash Company                  8922
The Best of Everything            8923
Length: 8924, dtype: int64

In [17]:
# Function that takes in movie title as input and gives recommendations 
def content_recommender(title, cosine_sim=cosine_sim, df=df, indices=indices):
    # Obtain the index of the movie that matches the title
    idx = indices[title] 
    #idx = indices[title][0]#chi lay duy nhat the loai dau tien cua phim thoi

    # Get the pairwsie similarity scores of all movies with that movie
    # And convert it into a list of tuples as described above
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the cosine similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df['title'].iloc[movie_indices]

In [18]:
content_recommender('Penny Serenade')

6068                      Parineeta
2603                        Ventoux
1095       History Is Made at Night
4576              Friends with Kids
7768            Becoming Cary Grant
1485               To Catch a Thief
8086    Joe Dirt 2: Beautiful Loser
5957                       Daylight
4372                 Love Forbidden
1919        Night Across the Street
Name: title, dtype: object

In [19]:
content_recommender('Revenge')

7403    The Widow From Chicago
8431              Madrid, 1987
7639               Class Enemy
2327                Dark Horse
4787                   Imagine
5873              Maximum Ride
277      Kaho Naa... Pyaar Hai
1128        Female Perversions
8823             Prime Suspect
4594    Rachel Getting Married
Name: title, dtype: object

In [ ]:
#Get recommendations for The Lion King
#content_recommender('The Lion King')

### Explain

In [101]:
title='Penny Serenade'
cosine_sim=cosine_sim
df=df 
indices=indices

# Obtain the index of the movie that matches the title
idx = indices[title]
print('idx: ',idx)

# Get the pairwsie similarity scores of all movies with that movie
# And convert it into a list of tuples as described above (movie id, score)
sim_scores = list(enumerate(cosine_sim[idx])) 
print('sim_scores:\n ',sim_scores)
print('len',len(sim_scores))
# Sort the movies based on the cosine similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
print('sim_scores sort \n',sim_scores)


# Get the scores of the 10 most similar movies. Ignore the first movie.
sim_scores = sim_scores[1:11]
print('sim_scores top 10: \n ',sim_scores)

# Get the movie indices
movie_indices = [i[0] for i in sim_scores]
print('movie_indices: \n',movie_indices)

# Return the top 10 most similar movies
print('Return the top 10 most similar movies\n',df['title'].iloc[movie_indices])

idx:  3
sim_scores:
  [(0, 0.0), (1, 0.0), (2, 0.01855348312929148), (3, 1.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.02432263971085088), (10, 0.01410012088747418), (11, 0.0), (12, 0.0), (13, 0.011606312292785676), (14, 0.0), (15, 0.013647205985133222), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.028787728086324918), (24, 0.0), (25, 0.0), (26, 0.0), (27, 0.0), (28, 0.03181840454803833), (29, 0.0), (30, 0.00971415495625801), (31, 0.00879558226948117), (32, 0.0), (33, 0.0), (34, 0.029029564211385864), (35, 0.0), (36, 0.0), (37, 0.0141052787896774), (38, 0.01531525314215607), (39, 0.01436773111555062), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.0), (45, 0.0), (46, 0.0), (47, 0.02020552251518315), (48, 0.010816603345482078), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.0), (53, 0.0), (54, 0.0), (55, 0.0), (56, 0.0), (57, 0.012672253740594669), (58, 0.0), (59, 0.005340134959200259), (60, 0.007531672388378357), (61, 0.02887535413237

In [103]:
title='Revenge'
cosine_sim=cosine_sim
df=df 
indices=indices

# Obtain the index of the movie that matches the title
idx = indices[title]
#idx = indices[title][0] # chỉ lấy thể loại đầu tiên của phim tho6i
print(type(idx))
print('idx: ',idx)

# Get the pairwsie similarity scores of all movies with that movie
# And convert it into a list of tuples as described above (movie id, score)
sim_scores = list(enumerate(cosine_sim[idx])) 
print('sim_scores:\n ',sim_scores)
print('len',len(sim_scores))
# Sort the movies based on the cosine similarity scores
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
print('sim_scores sort \n',sim_scores)


# Get the scores of the 10 most similar movies. Ignore the first movie.
sim_scores = sim_scores[1:11]
print('sim_scores top 10: \n ',sim_scores)

# Get the movie indices
movie_indices = [i[0] for i in sim_scores]
print('movie_indices: \n',movie_indices)

# Return the top 10 most similar movies
print('Return the top 10 most similar movies\n',df['title'].iloc[movie_indices])

<class 'numpy.int64'>
idx:  8661
sim_scores:
  [(0, 0.0), (1, 0.0), (2, 0.0187070256585414), (3, 0.0), (4, 0.0), (5, 0.0), (6, 0.0), (7, 0.0), (8, 0.0), (9, 0.0), (10, 0.05876186187024271), (11, 0.048413747001700154), (12, 0.0), (13, 0.0), (14, 0.0), (15, 0.013546264311781879), (16, 0.0), (17, 0.0), (18, 0.0), (19, 0.006852454725043154), (20, 0.0), (21, 0.0), (22, 0.0), (23, 0.0), (24, 0.0), (25, 0.05041215108365122), (26, 0.0), (27, 0.019131563960357974), (28, 0.0), (29, 0.0), (30, 0.0), (31, 0.0), (32, 0.0), (33, 0.019429652714530423), (34, 0.0), (35, 0.053152087964510215), (36, 0.0), (37, 0.016092523604924802), (38, 0.026002831143472765), (39, 0.0), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.0), (44, 0.013150397773562023), (45, 0.0), (46, 0.05815332937805098), (47, 0.008540950430582386), (48, 0.019986420064685068), (49, 0.0), (50, 0.0), (51, 0.0), (52, 0.00810118385445327), (53, 0.06982802317553505), (54, 0.011145538612264095), (55, 0.0), (56, 0.053082567611596326), (57, 0.0), (58, 0.0

### End explain

# Metadata Based Recommender

In [21]:
# Load the keywords and credits files
cred_df = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/credits.csv')
key_df = pd.read_csv('/content/drive/My Drive/Data/Movie dataset/keywords.csv')

In [22]:
#Print the head of the credit dataframe
cred_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [23]:
#Print the head of the keywords dataframe
key_df.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
#Convert the IDs of df into int
#df['id'] = df['id'].astype('int')

In [52]:
not_convert_list=[]
for i in range(len(df)):
  try:
    df['id'][i]=int(df['id'][i])
  except:
    print('cannot convert: ',df['id'][i])
    not_convert_list.append(df.index[i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


cannot convert:  2012-09-29


In [55]:
df=df[~df.index.isin(not_convert_list) ]
df.shape

(8923, 8)

In [60]:
df.id=df.id.astype('int')
df.id

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0        43649
1        47212
2       133953
3        43795
4        23981
         ...  
8919    385736
8920      1576
8921     43316
8922     37822
8923     57103
Name: id, Length: 8923, dtype: int64

In [61]:
# Function to convert all non-integer IDs to NaN
def clean_ids(x):
    try:
        return int(x)
    except:
        return np.nan

In [62]:
#Clean the ids of df
df['id'] = df['id'].apply(clean_ids)

#Filter all rows that have a null ID
df = df[df['id'].notnull()]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [63]:
# Convert IDs into integer
df['id'] = df['id'].astype('int')
key_df['id'] = key_df['id'].astype('int')
cred_df['id'] = cred_df['id'].astype('int')

# Merge keywords and credits into your main metadata dataframe
df = df.merge(cred_df, on='id')
df = df.merge(key_df, on='id')

#Display the head of df
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,The New Comics,['comedy'],102.0,5.0,32.0,1994,A few funny little novels about different aspe...,43649,"[{'cast_id': 5, 'character': 'Paolo', 'credit_...","[{'credit_id': '52fe4657c3a36847f80f9b29', 'de...",[]
1,China Girl,"['romance', 'crime', 'drama']",90.0,6.6,8.0,1987,Teenage lovers Tony (Richard Panebianco) and T...,47212,"[{'cast_id': 2, 'character': 'Alby', 'credit_i...","[{'credit_id': '52fe4725c3a36847f8125cd5', 'de...","[{'id': 497, 'name': 'shakespeare'}, {'id': 54..."
2,Snow Shark: Ancient Snow Beast,"['science fiction', 'horror']",80.0,4.0,2.0,2011,12 years ago during a scientific expedition 3 ...,133953,"[{'cast_id': 2, 'character': 'Mike Evans', 'cr...","[{'credit_id': '542633270e0a2659f3000a33', 'de...","[{'id': 1299, 'name': 'monster'}, {'id': 2988,..."
3,Penny Serenade,"['drama', 'romance']",119.0,6.4,23.0,1941,A light-hearted romantic drama starring Cary G...,43795,"[{'cast_id': 1, 'character': 'Julie Gardiner A...","[{'credit_id': '52fe465fc3a36847f80fb6cb', 'de...","[{'id': 2676, 'name': 'honeymoon'}, {'id': 509..."
4,Iron And Silk,"['drama', 'comedy']",92.0,6.0,1.0,1991,Iron and Silk is a 1990 movie based on the epo...,23981,"[{'cast_id': 1002, 'character': 'Teacher Mark'...","[{'credit_id': '5998ab5fc3a3680f4e00b626', 'de...","[{'id': 779, 'name': 'martial arts'}, {'id': 7..."


In [64]:
# Convert the stringified objects into the native python objects
from ast import literal_eval

features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    df[feature] = df[feature].apply(literal_eval)

In [66]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords
0,The New Comics,[comedy],102.0,5.0,32.0,1994,A few funny little novels about different aspe...,43649,"[{'cast_id': 5, 'character': 'Paolo', 'credit_...","[{'credit_id': '52fe4657c3a36847f80f9b29', 'de...",[]
1,China Girl,"[romance, crime, drama]",90.0,6.6,8.0,1987,Teenage lovers Tony (Richard Panebianco) and T...,47212,"[{'cast_id': 2, 'character': 'Alby', 'credit_i...","[{'credit_id': '52fe4725c3a36847f8125cd5', 'de...","[{'id': 497, 'name': 'shakespeare'}, {'id': 54..."
2,Snow Shark: Ancient Snow Beast,"[science fiction, horror]",80.0,4.0,2.0,2011,12 years ago during a scientific expedition 3 ...,133953,"[{'cast_id': 2, 'character': 'Mike Evans', 'cr...","[{'credit_id': '542633270e0a2659f3000a33', 'de...","[{'id': 1299, 'name': 'monster'}, {'id': 2988,..."
3,Penny Serenade,"[drama, romance]",119.0,6.4,23.0,1941,A light-hearted romantic drama starring Cary G...,43795,"[{'cast_id': 1, 'character': 'Julie Gardiner A...","[{'credit_id': '52fe465fc3a36847f80fb6cb', 'de...","[{'id': 2676, 'name': 'honeymoon'}, {'id': 509..."
4,Iron And Silk,"[drama, comedy]",92.0,6.0,1.0,1991,Iron and Silk is a 1990 movie based on the epo...,23981,"[{'cast_id': 1002, 'character': 'Teacher Mark'...","[{'credit_id': '5998ab5fc3a3680f4e00b626', 'de...","[{'id': 779, 'name': 'martial arts'}, {'id': 7..."


In [67]:
#Print the first cast member of the first movie in df
df.iloc[0]['crew'][0]

{'credit_id': '52fe4657c3a36847f80f9b29',
 'department': 'Directing',
 'gender': 0,
 'id': 106345,
 'job': 'Director',
 'name': 'Neri Parenti',
 'profile_path': '/bQMjLbk5gJngFFlx1SD0yorkhRZ.jpg'}

In [70]:
# Extract the director's name. If director is not listed, return NaN
def get_director(x):
    for crew_member in x:
        if crew_member['job'] == 'Director':
            return crew_member['name']
    return np.nan

In [71]:
#Define the new director feature
df['director'] = df['crew'].apply(get_director)

#Print the directors of the first five movies
df['director'].head()

0      Neri Parenti
1      Abel Ferrara
2      Sam Qualiana
3    George Stevens
4       Shirley Sun
Name: director, dtype: object

In [72]:
# Returns the list top 3 elements or entire list; whichever is more.
def generate_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [73]:
#Apply the generate_list function to cast and keywords
df['cast'] = df['cast'].apply(generate_list)
df['keywords'] = df['keywords'].apply(generate_list)

In [74]:
#Only consider a maximum of 3 genres
df['genres'] = df['genres'].apply(lambda x: x[:3])

In [75]:
# Print the new features of the first 5 movies along with title
df[['title', 'cast', 'director', 'keywords', 'genres']].head()

,title,cast,director,keywords,genres
0,The New Comics,"[Paolo Villaggio, Renato Pozzetto, Ramona Bade...",Neri Parenti,[],[comedy]
1,China Girl,"[James Russo, Richard Panebianco, David Caruso]",Abel Ferrara,"[shakespeare, street gang, chinatown]","[romance, crime, drama]"
2,Snow Shark: Ancient Snow Beast,"[Sam Qualiana, Michael O'Hear, Jackey Hall]",Sam Qualiana,"[monster, shark attack]","[science fiction, horror]"
3,Penny Serenade,"[Irene Dunne, Cary Grant, Beulah Bondi]",George Stevens,"[honeymoon, sterility, daughter]","[drama, romance]"
4,Iron And Silk,"[Mark Salzman, Pan Qingfu, Vivian Wu]",Shirley Sun,"[martial arts, kung fu, based on novel]","[drama, comedy]"


In [76]:
# Function to sanitize data to prevent ambiguity. It removes spaces and converts to lowercase
def sanitize(x):
    if isinstance(x, list):
        #Strip spaces and convert to lowercase
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [77]:
#Apply the generate_list function to cast, keywords, director and genres
for feature in ['cast', 'director', 'genres', 'keywords']:
    df[feature] = df[feature].apply(sanitize)

In [78]:
#Function that creates a soup out of the desired metadata
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

In [79]:
# Create the new soup feature
df['soup'] = df.apply(create_soup, axis=1)

In [80]:
#Display the soup of the first movie
df.iloc[0]['soup']

' paolovillaggio renatopozzetto ramonabadescu neriparenti comedy'

In [84]:
#Display the soup of the first movie
df.iloc[1]['soup']

'shakespeare streetgang chinatown jamesrusso richardpanebianco davidcaruso abelferrara romance crime drama'

In [83]:
df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id,cast,crew,keywords,director,soup
0,The New Comics,[comedy],102.0,5.0,32.0,1994,A few funny little novels about different aspe...,43649,"[paolovillaggio, renatopozzetto, ramonabadescu]","[{'credit_id': '52fe4657c3a36847f80f9b29', 'de...",[],neriparenti,paolovillaggio renatopozzetto ramonabadescu n...
1,China Girl,"[romance, crime, drama]",90.0,6.6,8.0,1987,Teenage lovers Tony (Richard Panebianco) and T...,47212,"[jamesrusso, richardpanebianco, davidcaruso]","[{'credit_id': '52fe4725c3a36847f8125cd5', 'de...","[shakespeare, streetgang, chinatown]",abelferrara,shakespeare streetgang chinatown jamesrusso ri...
2,Snow Shark: Ancient Snow Beast,"[sciencefiction, horror]",80.0,4.0,2.0,2011,12 years ago during a scientific expedition 3 ...,133953,"[samqualiana, michaelo'hear, jackeyhall]","[{'credit_id': '542633270e0a2659f3000a33', 'de...","[monster, sharkattack]",samqualiana,monster sharkattack samqualiana michaelo'hear ...
3,Penny Serenade,"[drama, romance]",119.0,6.4,23.0,1941,A light-hearted romantic drama starring Cary G...,43795,"[irenedunne, carygrant, beulahbondi]","[{'credit_id': '52fe465fc3a36847f80fb6cb', 'de...","[honeymoon, sterility, daughter]",georgestevens,honeymoon sterility daughter irenedunne carygr...
4,Iron And Silk,"[drama, comedy]",92.0,6.0,1.0,1991,Iron and Silk is a 1990 movie based on the epo...,23981,"[marksalzman, panqingfu, vivianwu]","[{'credit_id': '5998ab5fc3a3680f4e00b626', 'de...","[martialarts, kungfu, basedonnovel]",shirleysun,martialarts kungfu basedonnovel marksalzman pa...


In [85]:
# Import CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#Define a new CountVectorizer object and create vectors for the soup
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df['soup'])

In [86]:
count_matrix.shape

(9159, 25672)

In [88]:
count_matrix.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
#Import cosine_similarity function
from sklearn.metrics.pairwise import cosine_similarity

#Compute the cosine similarity score (equivalent to dot product for tf-idf vectors)
cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

In [ ]:
# Reset index of your df and construct reverse mapping again
df = df.reset_index()
indices2 = pd.Series(df.index, index=df['title'])

In [ ]:
content_recommender('The Lion King', cosine_sim2, df, indices2)

29607                                          Cheburashka
40904                   VeggieTales: Josh and the Big Wall
40913    VeggieTales: Minnesota Cuke and the Search for...
27768                                 The Little Matchgirl
15209             Spiderman: The Ultimate Villain Showdown
16613                            Cirque du Soleil: Varekai
24654                                  The Seventh Brother
29198                                      Superstar Goofy
30244                                              My Love
31179                Pokémon: Arceus and the Jewel of Life
Name: title, dtype: object